### Analysis


In [1]:
import numpy as np
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [2]:
df = pd.read_csv('./exported_data_2018-09-24_to_2023-07-14_V1.6.csv')
df.head()

,Variedad,Mercado,Unidad,Dia,Precio,Fecha,Volumen,Producto,Latitude,Longitude,Numero_Unidad,Nombre_Unidad,Volumen_Unidad,Precio_Unidad
0,Sin especificar,Femacal de La Calera,MALLA 18 KILOS,Lunes,6777.7759,2018-09-24,180.0,Cebolla,-32.785623,-71.188495,18,KILOS,3240.0,376.543106
1,Sin especificar,Central Lo Valledor de Santiago,MALLA 16 KILOS,Lunes,8830.4307,2018-09-24,2300.0,Cebolla,-33.481183,-70.682510,16,KILOS,36800.0,551.901919
2,Sin especificar,Terminal La Palmera de La Serena,MALLA 16 KILOS,Lunes,8950.0019,2018-09-24,1140.0,Cebolla,-29.890290,-71.253121,16,KILOS,18240.0,559.375119
3,Sin especificar,Vega Central Mapocho de Santiago,MALLA 18 KILOS,Martes,10727.2669,2018-09-25,550.0,Cebolla,-33.427612,-70.649499,18,KILOS,9900.0,595.959272
4,Sin especificar,Central Lo Valledor de Santiago,MALLA 16 KILOS,Martes,8732.1486,2018-09-25,2800.0,Cebolla,-33.481183,-70.682510,16,KILOS,44800.0,545.759287


In [3]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [9]:
# Remove data with few datapoints
df = df[~df['Mercado'].str.contains('Central Lo Valledor de Santiago')]

In [32]:
variedad_uva = ['Superior Seedless', 'Flame Seedless', 'Red Globe']
uvas = df[(df['Producto'] == 'Uva') & ((df['Mercado'] == 'Terminal La Palmera de La Serena') | (df['Mercado'] == 'Macroferia Regional de Talca')) & (df['Variedad'].isin(variedad_uva))]
px.line(uvas, x='Fecha', y='Precio_Unidad', color='Variedad')

In [33]:
#a = df.copy()
a = uvas.copy()
a['Week_Start_Date'] = a['Fecha'] - pd.to_timedelta(a['Fecha'].dt.dayofweek, unit='D')
a['Week_End_Date'] = a['Week_Start_Date'] + pd.DateOffset(days=6)
result = a.groupby(['Producto', 'Mercado', 'Week_Start_Date', 'Latitude', 'Longitude', 'Variedad']).agg({'Precio_Unidad': np.mean}).reset_index()
result.sample(10)

,Producto,Mercado,Week_Start_Date,Latitude,Longitude,Variedad,Precio_Unidad
204,Uva,Terminal La Palmera de La Serena,2021-02-01,-29.890290,-71.253121,Superior Seedless,597.222222
50,Uva,Macroferia Regional de Talca,2020-06-15,-35.420813,-71.635581,Red Globe,687.500000
242,Uva,Terminal La Palmera de La Serena,2022-04-18,-29.890290,-71.253121,Red Globe,513.888889
37,Uva,Macroferia Regional de Talca,2020-01-13,-35.420813,-71.635581,Superior Seedless,1000.000000
55,Uva,Macroferia Regional de Talca,2020-07-27,-35.420813,-71.635581,Red Globe,1031.250000
82,Uva,Macroferia Regional de Talca,2021-02-08,-35.420813,-71.635581,Red Globe,666.666667
156,Uva,Macroferia Regional de Talca,2023-01-02,-35.420813,-71.635581,Superior Seedless,1400.000000
81,Uva,Macroferia Regional de Talca,2021-02-01,-35.420813,-71.635581,Superior Seedless,634.263889
25,Uva,Macroferia Regional de Talca,2019-11-25,-35.420813,-71.635581,Superior Seedless,2300.000000
233,Uva,Terminal La Palmera de La Serena,2022-02-21,-29.890290,-71.253121,Flame Seedless,430.555556


In [34]:
result['Mercado'].value_counts()

Mercado
Macroferia Regional de Talca        187
Terminal La Palmera de La Serena     70
Name: count, dtype: int64

In [21]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19371 entries, 0 to 19370
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Producto         19371 non-null  object        
 1   Mercado          19371 non-null  object        
 2   Week_Start_Date  19371 non-null  datetime64[ns]
 3   Precio_Unidad    19371 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 605.5+ KB


In [35]:
line_chart = px.line(result, x='Week_Start_Date', y='Precio_Unidad', color='Producto', line_group='Producto',
                     facet_col='Mercado', labels={'Week_Start_Date': 'Semana', 'Precio_Unidad': 'Precio'})
line_chart.update_layout(title='Line Chart Faceted by Continent', xaxis_title='Year', yaxis_title='GDP per Capita')
line_chart.show()

In [36]:
result.to_csv('./preciospormercado-uvas.csv', index=None)